In [1]:
# Group: 28
# Authors: Thieme Brandwagt and Sam Buisman
#=================================
# Assignment 1 - Aircraft Routing Problem 
#=================================
#%% ----------------------------------------------------------------------------------
from gurobipy import *
from openpyxl import *
from time import *
from sklearn.linear_model import LinearRegression
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import time
from folium import PolyLine
from folium.plugins import MarkerCluster

Find the data

In [2]:
file_path_demand = "DATA/Group28.xlsx"
file_path_airport = "DATA/AirportData.xlsx"
file_path_fleet = "DATA/FleetType.xlsx"

In [56]:
time_intervals = ["00:00-04:00", "04:00-08:00", "08:00-12:00", 
                  "12:00-16:00", "16:00-20:00", "20:00-00:00"]

In [57]:
demand_data = pd.read_excel(file_path_demand, usecols="D:AG", skiprows=4)
days = len(demand_data.columns) // len(time_intervals)
demand_data.columns = [f"Day {day+1} {interval}" 
                       for day in range(days) 
                       for interval in time_intervals]
demand_data

,Day 1 00:00-04:00,Day 1 04:00-08:00,Day 1 08:00-12:00,Day 1 12:00-16:00,Day 1 16:00-20:00,Day 1 20:00-00:00,Day 2 00:00-04:00,Day 2 04:00-08:00,Day 2 08:00-12:00,Day 2 12:00-16:00,...,Day 4 08:00-12:00,Day 4 12:00-16:00,Day 4 16:00-20:00,Day 4 20:00-00:00,Day 5 00:00-04:00,Day 5 04:00-08:00,Day 5 08:00-12:00,Day 5 12:00-16:00,Day 5 16:00-20:00,Day 5 20:00-00:00
0,0,0,0,0,0,0.000000,0.000000,0.000000,0,0,...,0,0,0,0.000000,0.000000,0.000000,0,0,0,0
1,0,0,0,0,0,99813.040063,45910.232511,15957.400559,0,0,...,0,0,0,58862.688998,3329.266303,38688.610701,0,0,0,0
2,0,0,0,0,0,43600.538164,293895.414002,7918.685016,0,0,...,0,0,0,47164.194059,10067.411228,20206.392817,0,0,0,0
3,0,0,0,0,0,85490.948822,108937.164821,4212.698805,0,0,...,0,0,0,37587.675995,14561.735797,43855.383986,0,0,0,0
4,0,0,0,0,0,23378.963448,55935.437082,12388.987898,0,0,...,0,0,0,132449.125143,10478.994482,69641.779562,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0,0,0,0,0,8415.508688,4398.662609,6277.996129,0,0,...,0,0,0,32574.238219,9804.583228,29979.559165,0,0,0,0
396,0,0,0,0,0,6657.222061,10236.820448,9920.077161,0,0,...,0,0,0,4938.693795,32235.163192,8581.843263,0,0,0,0
397,0,0,0,0,0,36400.193118,7453.314241,8686.827745,0,0,...,0,0,0,15461.033818,29556.432131,25324.890161,0,0,0,0
398,0,0,0,0,0,19228.007698,11396.314893,760.284813,0,0,...,0,0,0,10188.494568,3279.408300,6948.311703,0,0,0,0
